# Customer Churn Prediction Analysis

This notebook provides a comprehensive analysis of customer churn prediction using the Telco Customer Churn dataset.

## Table of Contents
1. [Data Loading and Overview](#1-data-loading-and-overview)
2. [Exploratory Data Analysis](#2-exploratory-data-analysis)
3. [Data Preprocessing](#3-data-preprocessing)
4. [Feature Engineering](#4-feature-engineering)
5. [Model Training and Evaluation](#5-model-training-and-evaluation)
6. [Feature Importance Analysis](#6-feature-importance-analysis)
7. [Model Comparison](#7-model-comparison)
8. [Conclusions and Recommendations](#8-conclusions-and-recommendations)

## 1. Data Loading and Overview

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import xgboost as xgb
import lightgbm as lgb
import shap

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
# Note: Download the Telco Customer Churn dataset from Kaggle
# https://www.kaggle.com/blastchar/telco-customer-churn

try:
    df = pd.read_csv('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')
    print(f"Dataset loaded successfully!")
    print(f"Shape: {df.shape}")
except FileNotFoundError:
    print("Dataset not found. Please download the Telco Customer Churn dataset.")
    print("URL: https://www.kaggle.com/blastchar/telco-customer-churn")
    # Create sample data for demonstration
    df = pd.DataFrame({
        'customerID': range(1000),
        'gender': np.random.choice(['Male', 'Female'], 1000),
        'SeniorCitizen': np.random.choice([0, 1], 1000),
        'tenure': np.random.randint(1, 72, 1000),
        'MonthlyCharges': np.random.uniform(18, 118, 1000),
        'TotalCharges': np.random.uniform(18, 8000, 1000),
        'Churn': np.random.choice(['Yes', 'No'], 1000, p=[0.27, 0.73])
    })
    print("Using sample data for demonstration.")

# Display basic information
print("\nDataset Info:")
print(df.info())
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Dataset overview
print("Dataset Overview:")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print(f"\nColumn names: {list(df.columns)}")

# Check for missing values
print("\nMissing values:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Basic statistics
print("\nBasic Statistics:")
df.describe(include='all')

## 2. Exploratory Data Analysis

In [ ]:
# Churn distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Count plot
churn_counts = df['Churn'].value_counts()
axes[0].pie(churn_counts.values, labels=churn_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Churn Distribution')

# Bar plot
sns.countplot(data=df, x='Churn', ax=axes[1])
axes[1].set_title('Churn Count')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

print(f"Churn Rate: {(df['Churn'] == 'Yes').mean():.2%}")

In [ ]:
# Analyze categorical features
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('customerID')  # Remove ID column
if 'Churn' in categorical_cols:
    categorical_cols.remove('Churn')  # Remove target variable

# Create subplots for categorical features
n_cols = 3
n_rows = (len(categorical_cols) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes]

for i, col in enumerate(categorical_cols[:len(axes)]):
    if i < len(categorical_cols):
        churn_by_category = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
        churn_by_category.plot(kind='bar', ax=axes[i], rot=45)
        axes[i].set_title(f'Churn Rate by {col}')
        axes[i].set_ylabel('Percentage')
        axes[i].legend(title='Churn')
    else:
        axes[i].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze numerical features
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(numerical_cols[:4]):
    df.boxplot(column=col, by='Churn', ax=axes[i])
    axes[i].set_title(f'{col} by Churn Status')
    axes[i].set_xlabel('Churn')

plt.suptitle('')  # Remove automatic title
plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
# Convert categorical variables to numerical for correlation
df_encoded = df.copy()
le = LabelEncoder()

for col in df_encoded.select_dtypes(include=['object']).columns:
    if col != 'customerID':
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))

# Calculate correlation matrix
correlation_matrix = df_encoded.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

## 3. Data Preprocessing

In [ ]:
# Data cleaning and preprocessing
df_clean = df.copy()

# Remove customerID as it's not useful for prediction
if 'customerID' in df_clean.columns:
    df_clean = df_clean.drop('customerID', axis=1)

# Handle missing values
print("Missing values before cleaning:")
print(df_clean.isnull().sum())

# Convert TotalCharges to numeric (it might be stored as string)
if 'TotalCharges' in df_clean.columns:
    df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')
    
    # Fill missing TotalCharges with median
    df_clean['TotalCharges'].fillna(df_clean['TotalCharges'].median(), inplace=True)

print("\nMissing values after cleaning:")
print(df_clean.isnull().sum())

print(f"\nDataset shape after cleaning: {df_clean.shape}")

In [ ]:
# Encode categorical variables
df_processed = df_clean.copy()

# Binary encoding for Yes/No columns
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_cols:
    if col in df_processed.columns:
        df_processed[col] = df_processed[col].map({'Yes': 1, 'No': 0})

# Label encoding for other categorical columns
categorical_cols = df_processed.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Churn')  # Keep target variable for later

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col])
    label_encoders[col] = le

# Encode target variable
target_encoder = LabelEncoder()
df_processed['Churn'] = target_encoder.fit_transform(df_processed['Churn'])

print("Categorical encoding completed!")
print(f"\nProcessed dataset shape: {df_processed.shape}")
df_processed.head()

## 4. Feature Engineering

In [ ]:
# Feature engineering
df_features = df_processed.copy()

# Create new features
if 'tenure' in df_features.columns and 'MonthlyCharges' in df_features.columns:
    # Average monthly charges per tenure
    df_features['AvgChargesPerTenure'] = df_features['MonthlyCharges'] / (df_features['tenure'] + 1)
    
    # Tenure groups
    df_features['TenureGroup'] = pd.cut(df_features['tenure'], 
                                       bins=[0, 12, 24, 48, 72], 
                                       labels=['0-1 year', '1-2 years', '2-4 years', '4+ years'])
    df_features['TenureGroup'] = df_features['TenureGroup'].cat.codes

if 'MonthlyCharges' in df_features.columns:
    # Monthly charges groups
    df_features['ChargesGroup'] = pd.cut(df_features['MonthlyCharges'], 
                                        bins=[0, 35, 65, 95, 200], 
                                        labels=['Low', 'Medium', 'High', 'Very High'])
    df_features['ChargesGroup'] = df_features['ChargesGroup'].cat.codes

if 'TotalCharges' in df_features.columns and 'MonthlyCharges' in df_features.columns:
    # Ratio of total to monthly charges
    df_features['TotalToMonthlyRatio'] = df_features['TotalCharges'] / df_features['MonthlyCharges']

# Service count (count of additional services)
service_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                'TechSupport', 'StreamingTV', 'StreamingMovies']
existing_service_cols = [col for col in service_cols if col in df_features.columns]
if existing_service_cols:
    df_features['ServiceCount'] = df_features[existing_service_cols].sum(axis=1)

print("Feature engineering completed!")
print(f"New dataset shape: {df_features.shape}")
print(f"New features created: {set(df_features.columns) - set(df_processed.columns)}")

## 5. Model Training and Evaluation

In [ ]:
# Prepare data for modeling
X = df_features.drop('Churn', axis=1)
y = df_features['Churn']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale numerical features
scaler = StandardScaler()
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test_scaled[numerical_columns] = scaler.transform(X_test[numerical_columns])

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Target distribution in training set: {y_train.value_counts().to_dict()}")

In [ ]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=42, verbose=-1)
}

# Train and evaluate models
model_results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Use scaled data for Logistic Regression, original for tree-based models
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    # Cross-validation score
    if name == 'Logistic Regression':
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='roc_auc')
    else:
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    
    model_results[name] = {
        'model': model,
        'auc_score': auc_score,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"AUC Score: {auc_score:.4f}")
    print(f"CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print(f"Classification Report:")
    print(classification_report(y_test, y_pred))

In [ ]:
# Plot ROC curves
plt.figure(figsize=(12, 8))

for name, results in model_results.items():
    fpr, tpr, _ = roc_curve(y_test, results['probabilities'])
    plt.plot(fpr, tpr, label=f"{name} (AUC = {results['auc_score']:.3f})")

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.grid(True)
plt.show()

## 6. Feature Importance Analysis

In [ ]:
# Feature importance analysis
best_model_name = max(model_results.keys(), key=lambda k: model_results[k]['auc_score'])
best_model = model_results[best_model_name]['model']

print(f"Best performing model: {best_model_name}")
print(f"Best AUC Score: {model_results[best_model_name]['auc_score']:.4f}")

# Get feature importance
if hasattr(best_model, 'feature_importances_'):
    importance = best_model.feature_importances_
    feature_names = X.columns
    
    # Create feature importance dataframe
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    sns.barplot(data=importance_df.head(15), x='importance', y='feature')
    plt.title(f'Top 15 Feature Importances - {best_model_name}')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    print(importance_df.head(10))

elif hasattr(best_model, 'coef_'):
    # For logistic regression
    coef = np.abs(best_model.coef_[0])
    feature_names = X.columns
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': coef
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(data=importance_df.head(15), x='importance', y='feature')
    plt.title(f'Top 15 Feature Coefficients (Absolute) - {best_model_name}')
    plt.xlabel('Absolute Coefficient Value')
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    print(importance_df.head(10))

In [ ]:
# SHAP analysis for model interpretability
try:
    if best_model_name in ['Random Forest', 'XGBoost', 'LightGBM']:
        # Create SHAP explainer
        explainer = shap.TreeExplainer(best_model)
        shap_values = explainer.shap_values(X_test.iloc[:100])  # Use first 100 samples for speed
        
        if isinstance(shap_values, list):
            shap_values = shap_values[1]  # For binary classification, take positive class
        
        # Summary plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_test.iloc[:100], show=False)
        plt.title(f'SHAP Summary Plot - {best_model_name}')
        plt.tight_layout()
        plt.show()
        
        # Feature importance plot
        plt.figure(figsize=(12, 6))
        shap.summary_plot(shap_values, X_test.iloc[:100], plot_type="bar", show=False)
        plt.title(f'SHAP Feature Importance - {best_model_name}')
        plt.tight_layout()
        plt.show()
        
except Exception as e:
    print(f"SHAP analysis failed: {e}")
    print("Continuing without SHAP analysis...")

## 7. Model Comparison

In [ ]:
# Model comparison summary
comparison_df = pd.DataFrame({
    'Model': list(model_results.keys()),
    'AUC Score': [results['auc_score'] for results in model_results.values()],
    'CV Mean': [results['cv_mean'] for results in model_results.values()],
    'CV Std': [results['cv_std'] for results in model_results.values()]
})

comparison_df = comparison_df.sort_values('AUC Score', ascending=False)

print("Model Performance Comparison:")
print(comparison_df)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# AUC Score comparison
sns.barplot(data=comparison_df, x='AUC Score', y='Model', ax=axes[0])
axes[0].set_title('Model AUC Score Comparison')
axes[0].set_xlim(0, 1)

# CV Score comparison with error bars
axes[1].barh(comparison_df['Model'], comparison_df['CV Mean'], 
             xerr=comparison_df['CV Std'], capsize=5)
axes[1].set_title('Cross-Validation Score Comparison')
axes[1].set_xlabel('CV Mean Score')
axes[1].set_xlim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, (name, results) in enumerate(model_results.items()):
    cm = confusion_matrix(y_test, results['predictions'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'Confusion Matrix - {name}')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('Actual')

plt.tight_layout()
plt.show()

## 8. Conclusions and Recommendations

In [ ]:
# Generate insights and recommendations
print("=" * 60)
print("CUSTOMER CHURN PREDICTION ANALYSIS - CONCLUSIONS")
print("=" * 60)

print(f"\n1. BEST PERFORMING MODEL:")
print(f"   - Model: {best_model_name}")
print(f"   - AUC Score: {model_results[best_model_name]['auc_score']:.4f}")
print(f"   - Cross-validation: {model_results[best_model_name]['cv_mean']:.4f} (+/- {model_results[best_model_name]['cv_std']:.4f})")

print(f"\n2. DATASET INSIGHTS:")
print(f"   - Total customers analyzed: {len(df)}")
print(f"   - Overall churn rate: {(df['Churn'] == 'Yes').mean():.2%}")
print(f"   - Features analyzed: {len(X.columns)}")

if 'importance_df' in locals():
    print(f"\n3. KEY CHURN INDICATORS:")
    for i, row in importance_df.head(5).iterrows():
        print(f"   - {row['feature'].replace('_', ' ').title()}: {row['importance']:.4f}")

print(f"\n4. BUSINESS RECOMMENDATIONS:")
print(f"   - Focus retention efforts on high-risk customers identified by the model")
print(f"   - Implement proactive customer engagement strategies")
print(f"   - Monitor key indicators like tenure, contract type, and monthly charges")
print(f"   - Consider offering incentives for longer-term contracts")
print(f"   - Improve customer service for high-value, at-risk customers")

print(f"\n5. MODEL DEPLOYMENT:")
print(f"   - The {best_model_name} model is ready for production deployment")
print(f"   - Implement regular model retraining with new data")
print(f"   - Set up monitoring for model performance drift")
print(f"   - Create automated alerts for high-risk customers")

print("\n" + "=" * 60)

In [ ]:
# Save the best model and preprocessing objects
import joblib
import os

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save the best model
joblib.dump(best_model, f'../models/best_churn_model_{best_model_name.lower().replace(" ", "_")}.pkl')
joblib.dump(scaler, '../models/scaler.pkl')
joblib.dump(label_encoders, '../models/label_encoders.pkl')
joblib.dump(target_encoder, '../models/target_encoder.pkl')

# Save feature names
feature_names = list(X.columns)
joblib.dump(feature_names, '../models/feature_names.pkl')

print(f"Model artifacts saved successfully!")
print(f"- Best model: ../models/best_churn_model_{best_model_name.lower().replace(' ', '_')}.pkl")
print(f"- Scaler: ../models/scaler.pkl")
print(f"- Label encoders: ../models/label_encoders.pkl")
print(f"- Target encoder: ../models/target_encoder.pkl")
print(f"- Feature names: ../models/feature_names.pkl")

In [ ]:
# Create a sample prediction function
def predict_churn(customer_data, model=best_model, scaler=scaler, 
                  label_encoders=label_encoders, feature_names=feature_names):
    """
    Predict churn probability for a single customer
    
    Args:
        customer_data (dict): Customer features
        
    Returns:
        dict: Prediction results
    """
    # Create DataFrame from input
    df_input = pd.DataFrame([customer_data])
    
    # Apply same preprocessing
    # (This is a simplified version - in production, use the preprocessing pipeline)
    
    # Make prediction
    if best_model_name == 'Logistic Regression':
        prediction_proba = model.predict_proba(df_input)[:, 1][0]
    else:
        prediction_proba = model.predict_proba(df_input)[:, 1][0]
    
    prediction = 1 if prediction_proba > 0.5 else 0
    
    return {
        'churn_probability': prediction_proba,
        'churn_prediction': prediction,
        'risk_level': 'High' if prediction_proba > 0.7 else 'Medium' if prediction_proba > 0.3 else 'Low'
    }

# Example usage
sample_customer = {
    'tenure': 12,
    'MonthlyCharges': 75.0,
    'TotalCharges': 900.0
    # Add other features as needed
}

print("\nSample Prediction:")
print("Customer data:", sample_customer)
# result = predict_churn(sample_customer)
# print("Prediction result:", result)
print("(Note: Full prediction function requires complete preprocessing pipeline)")